In [ ]:
#Merge cre channel with nuclei segmentation

In [1]:
import numpy as np
import skimage
from skimage import img_as_ubyte, exposure
import cv2
import os
from os import listdir
from os.path import isfile, join
from PIL import Image
import fnmatch

In [15]:
bluePath = '/Users/jbj7388/Library/CloudStorage/GoogleDrive-jonasbraunnu@gmail.com/.shortcut-targets-by-id/1D_bPjhT95POOAirSmkzFskKPd4RlOOwP/LuisaLab/data/croppedImages/rainbow'
nucleiPath = '/Users/jbj7388/Library/CloudStorage/GoogleDrive-jonasbraunnu@gmail.com/.shortcut-targets-by-id/1D_bPjhT95POOAirSmkzFskKPd4RlOOwP/LuisaLab/data/segmentations/rainbow'
savePath = '/Users/jbj7388/Library/CloudStorage/GoogleDrive-jonasbraunnu@gmail.com/.shortcut-targets-by-id/1D_bPjhT95POOAirSmkzFskKPd4RlOOwP/LuisaLab/data/mergedImages/rainbow'

maturities = ['8mo']

In [4]:
nucleiFolders = [x[0] for x in os.walk(nucleiPath)]
nucleiFolders = sorted(nucleiFolders)

blueFolders = [x[0] for x in os.walk(bluePath)]
blueFolders = sorted(blueFolders)

nucleiFolders = fnmatch.filter(nucleiFolders, '*nuclei/*')
blueFolders = fnmatch.filter(blueFolders, '*orange/*')

pattern = "*notPerfect*"
foldersWrong = fnmatch.filter(nucleiFolders, pattern)
s = set(nucleiFolders) - set(foldersWrong)

In [23]:
for m in maturities:
        age = "*" + m + "*"
        nucleiFoldersAge = fnmatch.filter(s, age)
        blueFoldersAge = fnmatch.filter(blueFolders, age)
        for a in range(len(nucleiFoldersAge)):
                nucleifiles = [f for f in listdir(nucleiFoldersAge[a]) if isfile(join(nucleiFoldersAge[a], f))]
                aortaName = nucleiFoldersAge[a].split("/")[-1]
                bluefolder = fnmatch.filter(blueFoldersAge, '*/'+ aortaName)
                print(bluefolder)
                bluefiles = [f for f in listdir(bluefolder[0]) if isfile(join(bluefolder[0], f))]
                print(bluefiles)
                for i in range(len(bluefiles)):
                        patternBlue = "*-" + str(i+1) + '.tif'
                        patternNuclei = "*-" + str(i+1) + '_seg.npy'
                        blueFile = fnmatch.filter(bluefiles, patternBlue)
                        nucleiFile = fnmatch.filter(nucleifiles, patternNuclei)
                        blueImage = skimage.io.imread(join(bluefolder[0],blueFile[0]))
                        nuclei = np.load(join(nucleiFoldersAge[a],nucleiFile[0]), allow_pickle=True).item()
                        nucleiMasks = nuclei['masks']
                        nucleiMaskBlueInt = np.where(nucleiMasks > 0, blueImage, 0 )
                        nucleiMaskOnlyBlue = np.where(nucleiMaskBlueInt > np.quantile(blueImage, 0.95), nucleiMaskBlueInt, 0)
                        zeros = np.zeros(blueImage.shape, dtype="uint8")
                        dapi8bitNormalized= img_as_ubyte(exposure.rescale_intensity(nucleiMaskOnlyBlue))
                        blueChannel8bitNormalized = img_as_ubyte(exposure.rescale_intensity(blueImage))
                        merged8bitNormalized = cv2.merge([zeros, dapi8bitNormalized,blueChannel8bitNormalized])
                        im = Image.fromarray(merged8bitNormalized)
                        path_to_save = join(savePath, m, "orange")
                        os.chdir(path_to_save)
                        if not os.path.exists(aortaName):
                                os.makedirs(aortaName)
                        path_to_save = join(savePath, m, "orange", aortaName)
                        filename = 'mergedOrangeNuclei95' + str(i+1) + '.tif'
                        im.save(join(path_to_save,filename))
                        print("Done!")

['/Users/jbj7388/Library/CloudStorage/GoogleDrive-jonasbraunnu@gmail.com/.shortcut-targets-by-id/1D_bPjhT95POOAirSmkzFskKPd4RlOOwP/LuisaLab/data/croppedImages/rainbow/8mo/orange/BVRB459']
['Cropped_IMG-8.tif', 'Cropped_IMG-12.tif', 'Cropped_IMG-13.tif', 'Cropped_IMG-9.tif', 'Cropped_IMG-11.tif', 'Cropped_IMG-10.tif', 'Cropped_IMG-14.tif', 'Cropped_IMG-15.tif', 'Cropped_IMG-16.tif', 'Cropped_IMG-1.tif', 'Cropped_IMG-2.tif', 'Cropped_IMG-3.tif', 'Cropped_IMG-7.tif', 'Cropped_IMG-6.tif', 'Cropped_IMG-4.tif', 'Cropped_IMG-5.tif']


ValueError: operands could not be broadcast together with shapes (488,506) (112,556) () 